In [5]:
!pip install seaborn

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns


In [7]:
import tensorflow as tf

import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from absl import logging
import matplotlib.pyplot as plt
import os
import re
import seaborn as sns



In [8]:
df = pd.read_csv("ignite.csv")
df.head()

,Unnamed: 0,Algorithms and Data Structures,Introduction to Java Programming,Software Construction:\nObject Orientated Design,Data Abstraction,Introduction to Piano - By PGN Piano!,Introduction to Gypsy Jazz Guitar,Walking Bass - Quick Start Guide,Blues Bass - Quick Start Guide,Learn to trade Chart Patterns,Ultimate Investment Banking Course,Financial Modeling for Business Analysts and Consultants,Financial Modeling and Valuation: Complete Beginner to Pro,How To Build Your Own Web Banner Design Business,Canva: Design and Sell Your Own Professional Canva Designs,Design your own Social Media Logo Icons in Adobe Illustrator,Advance Design of Logos and Brand Identity
0,1,9.0,3.0,4.0,NaN,7.0,9.0,NaN,5.0,5.0,9.0,9.0,2.0,4.0,5.0,4.0,9.0
1,2,NaN,NaN,9.0,NaN,8.0,6.0,6.0,NaN,4.0,6.0,4.0,5.0,5.0,6.0,6.0,5.0
2,3,4.0,8.0,NaN,7,6.0,7.0,8.0,4.0,NaN,3.0,5.0,7.0,6.0,3.0,NaN,2.0
3,4,6.0,4.0,6.0,NaN,NaN,8.0,4.0,6.0,4.0,8.0,7.0,4.0,3.0,8.0,4.0,7.0
4,5,NaN,NaN,NaN,8,NaN,7.0,NaN,8.0,1.0,5.0,2.0,8.0,7.0,1.0,6.0,5.0


In [9]:
df.tail()

,Unnamed: 0,Algorithms and Data Structures,Introduction to Java Programming,Software Construction:\nObject Orientated Design,Data Abstraction,Introduction to Piano - By PGN Piano!,Introduction to Gypsy Jazz Guitar,Walking Bass - Quick Start Guide,Blues Bass - Quick Start Guide,Learn to trade Chart Patterns,Ultimate Investment Banking Course,Financial Modeling for Business Analysts and Consultants,Financial Modeling and Valuation: Complete Beginner to Pro,How To Build Your Own Web Banner Design Business,Canva: Design and Sell Your Own Professional Canva Designs,Design your own Social Media Logo Icons in Adobe Illustrator,Advance Design of Logos and Brand Identity
46,47,8.0,4.0,6.0,9,NaN,NaN,5.0,8.0,5.0,5.0,NaN,7.0,9.0,4.0,5.0,9.0
47,48,7.0,8.0,9.0,6,4.0,1.0,3.0,7.0,7.0,1.0,6.0,4.0,5.0,5.0,NaN,8.0
48,49,10.0,NaN,9.0,8,2.0,1.0,4.0,5.0,NaN,7.0,3.0,8.0,6.0,6.0,5.0,1.0
49,50,10.0,2.0,9.0,8,4.0,3.0,5.0,NaN,1.0,NaN,8.0,NaN,2.0,9.0,3.0,4.0
50,SUM,339.0,268.0,296.0,232,221.0,208.0,220.0,201.0,205.0,227.0,209.0,221.0,244.0,245.0,238.0,230.0


In [10]:
df_User_Interest = pd.read_csv("User_Interest.csv")

In [11]:
columns = [i for i in df_User_Interest.columns]
scores_all = []
for i in df_User_Interest.index:
  scores = [x for x in df_User_Interest.iloc[i] ]
  scores_all.append(scores)

sentences= [ ]

for scores in scores_all:
  sentence_indiv = ''
  for i in range(len(scores)):
    sentence_indiv += scores[i]*columns[i]+' '

  sentences.append(sentence_indiv)

In [12]:
df_User_Interest['sentences'] = sentences

In [13]:
df_User_Interest

,Identity,Maketing,Music & Art,Informtion Technology,Education,Business & Finance,Media & Internet,Engineering & Manufacturing,Healthcare,Science & Pharmaceuticals,Environment & Agriculture,sentences
0,1,2,0,10,1,9,3,4,5,6,7,Identity MaketingMaketing Informtion Technolo...
1,2,3,4,8,1,7,6,2,5,9,10,IdentityIdentity MaketingMaketingMaketing Musi...
2,3,1,3,5,2,4,7,6,8,10,9,IdentityIdentityIdentity Maketing Music & ArtM...
3,4,2,4,10,8,7,6,5,3,0,1,IdentityIdentityIdentityIdentity MaketingMaket...
4,5,0,0,10,8,9,6,4,2,1,3,IdentityIdentityIdentityIdentityIdentity Inf...
5,6,2,0,8,1,5,6,10,3,4,7,IdentityIdentityIdentityIdentityIdentityIdenti...
6,7,0,7,0,10,7,6,9,5,7,3,IdentityIdentityIdentityIdentityIdentityIdenti...
7,8,9,3,4,0,4,10,4,3,0,9,IdentityIdentityIdentityIdentityIdentityIdenti...
8,9,3,10,0,5,2,3,2,8,6,9,IdentityIdentityIdentityIdentityIdentityIdenti...
9,10,3,2,1,2,0,10,4,2,4,10,IdentityIdentityIdentityIdentityIdentityIdenti...


In [14]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [15]:
sentence_embeddings = embed(sentences)

In [16]:
coursenames = [i for i in df.columns]  
coursenames = coursenames[1:]
coursenames_embedding = embed(coursenames)

In [17]:
def cosine(u, v ):#Calculate cosine similarity between user interest sentence and course names
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [18]:
user_similarities_to_course = {}
n = 0
for x in sentence_embeddings:
  
  metrics = {}
  for y in range(len(coursenames_embedding)):
     cs = cosine(x ,  coursenames_embedding[y])
     metrics[coursenames[y]] = cs
  sorted_metrics_tuples = sorted(metrics.items(), key=lambda item: item[1] , reverse=True)
  top_3_similar_courses = [x[0] for x in sorted_metrics_tuples[:3]]

  user_similarities_to_course['user '+str(n)] = top_3_similar_courses
  n += 1


    



In [19]:
user_similarities_to_course

{'user 0': ['Data Abstraction ',
  'Introduction to Java Programming ',
  'Financial Modeling and Valuation: Complete Beginner to Pro'],
 'user 1': ['Data Abstraction ',
  'How To Build Your Own Web Banner Design Business',
  'Introduction to Java Programming '],
 'user 10': ['How To Build Your Own Web Banner Design Business',
  'Data Abstraction ',
  'Financial Modeling for Business Analysts and Consultants'],
 'user 11': ['Data Abstraction ',
  'How To Build Your Own Web Banner Design Business',
  'Introduction to Piano - By PGN Piano!'],
 'user 12': ['How To Build Your Own Web Banner Design Business',
  'Introduction to Piano - By PGN Piano!',
  'Canva: Design and Sell Your Own Professional Canva Designs'],
 'user 13': ['How To Build Your Own Web Banner Design Business',
  'Canva: Design and Sell Your Own Professional Canva Designs',
  'Financial Modeling and Valuation: Complete Beginner to Pro'],
 'user 14': ['Financial Modeling for Business Analysts and Consultants',
  'Canva: Des